## Convert Mick's movement data

1. Read CSV
2. Convert CSV to XML file
3. Join the two files together


- TODO: A little more "why" documentation - Aaron
- TODO: TT to get conda environment working again automatically and/or with local VS Code instance - testing of Gitpod.io YAML setup and GitHub.com "Spaces" equivalent
- TODO: TT - Need to check that both local and remote (in this instance Github.com) data modes work

In [3]:
#!pip install pandas

In [4]:
import pandas as pd
from pathlib import Path

In [5]:
# Define the location of the data source - either local or Cloud (e.g. Github for non-PII data)

# DATASOURCE_TYPE = "local"  # Local machine

DATASOURCE_TYPE = "gh"       # Github (gh) - Synthetic or non-PII data only

In [6]:
DATAFILE_PATH = "data"
DATAFILE_CSV  = "2021_09_10_15_09_Sassafras_Power_Climb,_Sunset,_South_Carolina.csv"
DATAFILE_URL_PATH = "https://raw.githubusercontent.com/DataBooth/client-youngm-ifit/main/data/" 
DATAFILE_URL = DATAFILE_URL_PATH + DATAFILE_CSV.replace(",", "%2C")

In [26]:
def set_local_or_remote_data_path(datasource_type, datafile_path, datafile_csv, datafile_url):
  if datasource_type ==  "local":
    datafile = Path(datafile_path) / datafile_csv
    print("Local datasource: " + datafile.as_posix())
    tcxfile = Path(datafile_path) / datafile_csv.replace("csv", "tcx")
    print("TCX file to be created: " + tcxfile.as_posix())
    return datafile, tcxfile
  elif datasource_type == "gh":
    datafile = datafile_url
    print("Remote datasource: " + datafile)
    tcxfile = Path(DATAFILE_PATH) / DATAFILE_CSV.replace("csv", "tcx")
    print("TCX file to be created: " + tcxfile.as_posix())
    return datafile, tcxfile
  else:
    print("ERROR: Unknown `datasource_type`")
  return None

In [29]:
datafile, tcxfile = set_local_or_remote_data_path(DATASOURCE_TYPE, DATAFILE_PATH, DATAFILE_CSV, DATAFILE_URL)

Remote datasource: https://raw.githubusercontent.com/DataBooth/client-youngm-ifit/main/data/2021_09_10_15_09_Sassafras_Power_Climb%2C_Sunset%2C_South_Carolina.csv
TCX file to be created: data/2021_09_10_15_09_Sassafras_Power_Climb,_Sunset,_South_Carolina.tcx


In [19]:
# How to read csv

data_df = pd.read_csv(datafile, skiprows=2)
print(data_df.head())

# How to read xml
# xml_data = open('tcxfile.tcx', 'r').read()
# print(xml_data)

    Time   Miles   MPH  Watts   HR  RPM  Resistance  Relative Resistance  \
0  00:00  0.0000  6.56     20  0.0   49           2                    8   
1  00:01  0.0021  6.56      1  0.0   49           2                    8   
2  00:02  0.0043  6.56      1  0.0   49           2                    8   
3  00:03  0.0064  6.56      2  0.0   49           2                    8   
4  00:04  0.0085  6.56      3  0.0   48           2                    8   

   Incline  
0      0.0  
1      0.0  
2      0.0  
3      0.0  
4      0.0  


In [20]:
data_df.rename(columns={"Relative Resistance": "RelativeResistance"}, inplace=True)

In [21]:
# TODO: Aaron to read about self-documenting code 

def convert_csv_row_to_xml(row):
    return """<Time>%s</Time>
    <Miles>%s</Miles>
    <MPH>%s</MPH>
    <Watts>%s</Watts>
    <HR>%s</HR>
    <RPM>%s</RPM>
    <Resistance>%s</Resistance>
    <Relative Resistance>%s</Relative Resistance>
    <Incline>%s</Incline>""" % (row.Time, row.Miles, row.MPH, row.Watts, row.HR, row.RPM, row.Resistance, row.RelativeResistance, row.Incline)

# TODO: The issue I currently have is how to change "Relative Resistance" to "RelativeResistance" for the above code to work

In [22]:
new_tcx = ''.join(data_df.apply(convert_csv_row_to_xml, axis=1))

In [24]:
#with open(tcxfile, "a+") as tcxwrite: 
#  for line in new_tcx:
#    tcxwrite.write(line)

## References

* https://towardsdatascience.com/the-easy-way-to-work-with-csv-json-and-xml-in-python-5056f9325ca9
* https://stackabuse.com/reading-and-writing-xml-files-in-python-with-pandas/
* https://roytuts.com/how-to-convert-csv-to-xml-using-python/
* https://stackoverflow.com/questions/41059264/simple-csv-to-xml-conversion-python